# How to...insert data into a database

This notebook shows how to use the entitymatching api to manipulate data into and from a local database. The API has the capability to create a database in a local folder, create tables and execute all the basic SQL operations (insert, delete, update and select).

In [1]:
# Sets up the location of the api relative to this notebook 
import sys
sys.path.append('../../../')

In [10]:
# Import the module for accessing a database
from esgmatching.dbmanager.SqlEngine import SqlEngine

## 1. Connect to a database

The string connection below creates a local database based on sqlite, that must be installed in the user's computer. Check the link https://www.tutorialspoint.com/sqlite/sqlite_installation.htm for more information on how to install and use sqlite. 

In [11]:
# Localization of the database to be created in relation to this jupyter notebook
# The database will be created in the /data/dabase folder, under the project main folder (EntityMatching)
path_db = '../../../data/database/'

In [12]:
# String connection used for sqlite. Others databases might require different information.
# In this example the connection is a combination of [sqlite statement] + [database path] + [database name]
str_connection = 'sqlite:///' + path_db + 'entitymatching.db'
str_connection

'sqlite:///../../../data/database/entitymatching.db'

In [13]:
# The database engine object is created by passing the string connection 
sqlengine_obj = SqlEngine(str_connection)

In [14]:
# The connect() method of the SqlEngine is used to stablish a connection with the database if it exists, 
# or to create a new one, otherwise. The parameter show_eco is False by default and indicates if the SQL statements 
# are echoed (or printed) in the default output channel. Therefore, let's set show_echo = True to see the Sql statements. 
sqlengine_obj.connect(show_echo=True)

In [15]:
# Check if the connection was stablished
sqlengine_obj.is_connected()

True

## 2. Create a table

In [16]:
# Prepare to create a table
table1_name = 'table1'
columns_names = ['isin', 'lei', 'company_name']

In [17]:
# The create_table method requires a name for table and a list with all column names and returns a table object. 
# The parameter add_idx creates an autoincrement column that will be the table's primary key.
table1_obj = sqlengine_obj.create_table(table1_name, columns_names, add_idx=True)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [15]:
# Check if the table was created successfully
is_table_created = sqlengine_obj.table_exists(table1_name)
is_table_created

2021-08-25 10:36:36,869 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-08-25 10:36:36,870 INFO sqlalchemy.engine.Engine [raw sql] ()


True

In [17]:
# Check the type of table object
type(table1_obj)

sqlalchemy.sql.schema.Table

## 3. Insert data in a table

In [18]:
# The data to be inserted is a dictionary in which the key is the column name and the value is its content.
data = {"isin": "SK1120005824", "lei": "097900BHK10000084115", "company_name": "CENTRAL PERK"}

In [19]:
# The insert_row() method adds the content to the table
sqlengine_obj.insert_row(table1_obj, data)

2021-08-25 10:37:33,604 INFO sqlalchemy.engine.Engine INSERT INTO table1 (isin, lei, company_name) VALUES (?, ?, ?)
2021-08-25 10:37:33,606 INFO sqlalchemy.engine.Engine [generated in 0.00128s] ('SK1120005824', '097900BHK10000084115', 'CENTRAL PERK')
2021-08-25 10:37:33,609 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
# It is possible to query the table and check the values in it
result = sqlengine_obj.query_table(table1_obj)
result

2021-08-25 10:37:38,979 INFO sqlalchemy.engine.Engine SELECT table1.idx, table1.isin, table1.lei, table1.company_name 
FROM table1
2021-08-25 10:37:38,980 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()


[(1, 'SK1120005824', '097900BHK10000084115', 'CENTRAL PERK')]

In [21]:
# The result of the query is a list of tupples. Therefore, recovering the values individually is an easy task
print('isin: {}'.format(result[0][1]))
print('lei: {}'.format(result[0][2]))
print('company_name: {}'.format(result[0][3]))

isin: SK1120005824
lei: 097900BHK10000084115
company_name: CENTRAL PERK


## 4. Insert multiple rows in a table

In [22]:
# To insert multiple rows, just add multiple dictionary of rows in a list and pass the list to the insert method 
data = []
data.append({"isin": "DE0005545503", "lei": "5299003VKVDCUPSS5X23", "company_name": "DUNDER MIFFLIN"})
data.append({"isin": "GB00B1YW4409", "lei": "254900B1P3S786KDAW57", "company_name": "HONEYDUKES"})

In [23]:
# The insert_row() method adds the multiple rows to the table
sqlengine_obj.insert_row(table1_obj, data)

2021-08-25 10:37:51,814 INFO sqlalchemy.engine.Engine INSERT INTO table1 (isin, lei, company_name) VALUES (?, ?, ?), (?, ?, ?)
2021-08-25 10:37:51,815 INFO sqlalchemy.engine.Engine [no key 0.00107s] ('DE0005545503', '5299003VKVDCUPSS5X23', 'DUNDER MIFFLIN', 'GB00B1YW4409', '254900B1P3S786KDAW57', 'HONEYDUKES')
2021-08-25 10:37:51,818 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
# It is possible to query the table and check the values in it
result = sqlengine_obj.query_table(table1_obj)
result

2021-08-25 10:37:58,599 INFO sqlalchemy.engine.Engine SELECT table1.idx, table1.isin, table1.lei, table1.company_name 
FROM table1
2021-08-25 10:37:58,599 INFO sqlalchemy.engine.Engine [cached since 19.62s ago] ()


[(1, 'SK1120005824', '097900BHK10000084115', 'CENTRAL PERK'),
 (2, 'DE0005545503', '5299003VKVDCUPSS5X23', 'DUNDER MIFFLIN'),
 (3, 'GB00B1YW4409', '254900B1P3S786KDAW57', 'HONEYDUKES')]

## 5. Select join...

In [26]:
# Prepare to create a table
table2_name = 'table2'
columns_names = ['isin', 'company_name']

In [27]:
# Create table2
table2_obj = sqlengine_obj.create_table(table2_name, columns_names, add_idx=True)

2021-08-25 10:38:33,938 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-25 10:38:33,940 INFO sqlalchemy.engine.Engine 
CREATE TABLE table2 (
	idx INTEGER NOT NULL, 
	isin VARCHAR, 
	company_name VARCHAR, 
	PRIMARY KEY (idx)
)


2021-08-25 10:38:33,941 INFO sqlalchemy.engine.Engine [no key 0.00137s] ()
2021-08-25 10:38:33,965 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
# To insert multiple rows, just add multiple dictionary of rows in a list and pass the list to the insert method 
data = []
data.append({"isin": "SK1120005824", "company_name": "CENTRAL PERK"})
data.append({"isin": "DE0005545503", "company_name": "DUNDER MIFFLIN"})

In [29]:
# The insert_row() method adds the multiple rows to the table
sqlengine_obj.insert_row(table2_obj, data)

2021-08-25 10:38:59,155 INFO sqlalchemy.engine.Engine INSERT INTO table2 (isin, company_name) VALUES (?, ?), (?, ?)
2021-08-25 10:38:59,156 INFO sqlalchemy.engine.Engine [no key 0.00098s] ('SK1120005824', 'CENTRAL PERK', 'DE0005545503', 'DUNDER MIFFLIN')
2021-08-25 10:38:59,159 INFO sqlalchemy.engine.Engine COMMIT


In [30]:
# Checking the values
result = sqlengine_obj.query_table(table2_obj)
result

2021-08-25 10:39:03,999 INFO sqlalchemy.engine.Engine SELECT table2.idx, table2.isin, table2.company_name 
FROM table2
2021-08-25 10:39:04,000 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ()


[(1, 'SK1120005824', 'CENTRAL PERK'), (2, 'DE0005545503', 'DUNDER MIFFLIN')]

In [36]:
for column in table2_obj.columns:
    print(type(column))

<class 'sqlalchemy.sql.schema.Column'>
<class 'sqlalchemy.sql.schema.Column'>
<class 'sqlalchemy.sql.schema.Column'>


In [ ]:
# Performing a join between the two tables
query = session.query(User, Document, DocumentsPermissions).join(Document).join(DocumentsPermissions)

## 6. Drop tables 

In [12]:
sqlengine_obj.drop_table(table1_obj)

2021-08-25 10:36:23,232 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-25 10:36:23,233 INFO sqlalchemy.engine.Engine 
DROP TABLE table1
2021-08-25 10:36:23,235 INFO sqlalchemy.engine.Engine [no key 0.00156s] ()
2021-08-25 10:36:23,250 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
sqlengine_obj.drop_table(table2_obj)

## 7. Disconnect from database

In [ ]:
sqlengine_obj.disconnect()

In [ ]:
sqlengine_obj.is_connected()